In [8]:
import json
import pandas as pd

from datetime import datetime
from modules.scoring import get_scoring_set, get_features, score, validate
from modules.training import split_data, transform_data

In [9]:
def display_fanta_squad(predictions, teams, index):
    fanta_team = predictions[predictions['fanta_team'] == teams[index]]
    return fanta_team, index+1    

In [10]:
try:
    
    ###############################################################################################
    status = 'Loading config ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    with open('config/config.json') as config:
        config = json.load(config)
    status = 'Loading config done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Getting scoring set ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    scoring_set = get_scoring_set(config['league_name'])
    scoring_set = get_features(scoring_set, config['league_name'])
    scoring_set = scoring_set[~scoring_set.home_team.isin(['Bologna', 'Verona', 'Lazio', 'Atalanta', 'Genoa', 'Udinese', 'Inter', 'Venezia'])]
    scoring_set = scoring_set[~scoring_set.away_team.isin(['Bologna', 'Verona', 'Lazio', 'Atalanta', 'Genoa', 'Udinese', 'Inter', 'Venezia'])]
    X_raw, metadata = split_data(scoring_set, metadata_columns=['fanta_team', 'date', 'role', 'player_id', 'player','team'], production=True)
    status = 'Getting scoring set done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################
    
    ###############################################################################################
    status = 'Processing features ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    X = transform_data(X_test_raw=X_raw, categorical_columns=['home_team', 'away_team', 'is_home', 'season', 'month', 'day', 'hour', 'week_day', 'position'], production=True)
    status = 'Processing features done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Scoring ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    predictions = score(X, scoring_set, config['league_name'], config['gameweek']['current'])
    status = 'Scoring done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Preparing predictions for display...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    predictions['player'] = predictions['player'].apply(lambda x: x.split(' ')[-1])
    status = 'Preparing predictions for display done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Updating config ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    config['gameweek']['current'] = config['gameweek']['current'] + 1
    with open('config/config.json', 'w') as f:
        json.dump(config, f, indent=4)
    status = 'Updating config done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

except Exception as e:
    print('Exception: {}'.format(e))
    print('Errored out during: {}'.format(status))

2022-01-23 10:24:44 -> Loading config ...
2022-01-23 10:24:44 -> Loading config done!
2022-01-23 10:24:44 -> Getting scoring set ...
100%|██████████| 174/174 [00:10<00:00, 16.29it/s]2022-01-23 10:28:29 -> Getting scoring set done!
2022-01-23 10:28:29 -> Processing features ...
2022-01-23 10:28:29 -> Processing features done!
2022-01-23 10:28:29 -> Scoring ...
2022-01-23 10:28:29 -> Scoring done!
2022-01-23 10:28:29 -> Preparing predictions for display...
2022-01-23 10:28:29 -> Preparing predictions for display done!
2022-01-23 10:28:29 -> Updating config ...
2022-01-23 10:28:29 -> Updating config done!

Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator OneHotEncoder from version 1.0.2 when using version 1.0.1. This m

In [11]:
_deepnote_run_altair(predictions, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false},"aggregate":"sum"},"y":{"field":"fanta_team","type":"nominal","sort":{"order":"descending","encoding":"x"},"scale":{"type":"linear","zero":true}},"color":{"field":"fanta_team","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [12]:
teams = sorted(list(predictions['fanta_team'].unique()))
index = 0

In [13]:
team, index = display_fanta_squad(predictions, teams, index)

In [14]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [15]:
team, index = display_fanta_squad(predictions, teams, index)

In [16]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [17]:
team, index = display_fanta_squad(predictions, teams, index)

In [18]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [19]:
team, index = display_fanta_squad(predictions, teams, index)

In [20]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [21]:
team, index = display_fanta_squad(predictions, teams, index)

In [22]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [23]:
team, index = display_fanta_squad(predictions, teams, index)

In [24]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [25]:
team, index = display_fanta_squad(predictions, teams, index)

In [26]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [27]:
team, index = display_fanta_squad(predictions, teams, index)

In [28]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [29]:
try:
    
    ###############################################################################################
    status = 'Loading config ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    with open('config/config.json') as config:
        config = json.load(config)
    status = 'Loading config done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Retrieving past week predictions ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    validation = validate(config['league_name'], config['gameweek']['past'])
    status = 'Retrieving past week predictions done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Preparing predictions for display...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    validation['player'] = validation['player'].apply(lambda x: x.split(' ')[-1])
    status = 'Preparing predictions for display done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Updating config ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    config['gameweek']['past'] = config['gameweek']['past'] + 1
    with open('config/config.json', 'w') as f:
        json.dump(config, f, indent=4)
    status = 'Updating config done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

except Exception as e:
    print('Exception: {}'.format(e))
    print('Errored out during: {}'.format(status))

2022-01-23 10:28:31 -> Loading config ...
2022-01-23 10:28:31 -> Loading config done!
2022-01-23 10:28:31 -> Retrieving past week predictions ...
Exception: [Errno 2] No such file or directory: 'data/outputs/prediction/fanta-camplus-citta-studi/gameweek_23.csv'
Errored out during: Retrieving past week predictions ...


In [30]:
teams = sorted(list(validation['fanta_team'].unique()))
index = 0

NameError: name 'validation' is not defined

In [ ]:
team, index = display_fanta_squad(validation, teams, index)

In [ ]:
team, index = display_fanta_squad(validation, teams, index)

In [ ]:
team, index = display_fanta_squad(validation, teams, index)

In [ ]:
team, index = display_fanta_squad(validation, teams, index)

In [ ]:
team, index = display_fanta_squad(validation, teams, index)

In [ ]:
team, index = display_fanta_squad(validation, teams, index)

In [ ]:
team, index = display_fanta_squad(validation, teams, index)

In [ ]:
team, index = display_fanta_squad(validation, teams, index)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=82f6a53d-2812-4762-aa0b-0b8fac2a34e3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>